In [14]:
%pip install natsort

596.08s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [25]:
import cv2
import numpy as np
from ultralytics import YOLO
import supervision as sv
import os
import glob
from natsort import natsorted

# Load YOLOv8 model
model = YOLO("yolov8n.pt")  # Consider 'yolov8s.pt' or 'yolov8m.pt' for higher accuracy

# Tracker initialization
tracker = sv.ByteTrack()

# Initialize annotators
box_annotator = sv.BoxAnnotator(thickness=2)
label_annotator = sv.LabelAnnotator(
    text_scale=0.5,
    text_thickness=1,
    text_padding=5
)

def process_image_sequence(images_folder, target_classes=['person'], confidence_threshold=0.3, save_output=True, output_folder="tracked_output", fps=30):
    # Collect and sort images naturally
    image_files = natsorted(glob.glob(os.path.join(images_folder, '*.*')))

    if not image_files:
        print(f"No images found in '{images_folder}'")
        return

    # Prepare output folder
    os.makedirs(output_folder, exist_ok=True)

    # Class IDs extraction
    class_ids = [class_id for class_id, name in model.names.items() if name in target_classes]

    for idx, image_path in enumerate(image_files):
        frame = cv2.imread(image_path)

        # YOLO inference
        results = model(frame, conf=confidence_threshold, classes=class_ids, verbose=False)[0]

        # Detections processing
        detections = sv.Detections.from_ultralytics(results)

        # Tracking
        tracked_detections = tracker.update_with_detections(detections)

        # Label formatting
        labels = [f"Player #{track_id}" for track_id in tracked_detections.tracker_id]

        # Annotation
        frame_with_boxes = box_annotator.annotate(scene=frame.copy(), detections=tracked_detections)
        annotated_frame = label_annotator.annotate(scene=frame_with_boxes, detections=tracked_detections, labels=labels)

        # Save annotated images
        if save_output:
            output_image_path = os.path.join(output_folder, f"frame_{idx + 1:05d}.jpg")
            cv2.imwrite(output_image_path, annotated_frame)

        # Progress update
        if idx % 50 == 0 or idx == len(image_files) - 1:
            print(f"Processed frame {idx + 1}/{len(image_files)}")

    print(f"Annotated frames saved to: {output_folder}")

if __name__ == "__main__":
    images_folder = "RBK_TDT17/1_train-val_1min_aalesund_from_start/img1"
    process_image_sequence(images_folder, target_classes=['person'], confidence_threshold=0.3, save_output=True)


Processed frame 1/1802
Processed frame 51/1802
Processed frame 101/1802
Processed frame 151/1802
Processed frame 201/1802
Processed frame 251/1802
Processed frame 301/1802
Processed frame 351/1802
Processed frame 401/1802
Processed frame 451/1802
Processed frame 501/1802
Processed frame 551/1802
Processed frame 601/1802
Processed frame 651/1802
Processed frame 701/1802
Processed frame 751/1802
Processed frame 801/1802
Processed frame 851/1802
Processed frame 901/1802
Processed frame 951/1802
Processed frame 1001/1802
Processed frame 1051/1802
Processed frame 1101/1802
Processed frame 1151/1802
Processed frame 1201/1802
Processed frame 1251/1802
Processed frame 1301/1802
Processed frame 1351/1802
Processed frame 1401/1802
Processed frame 1451/1802
Processed frame 1501/1802
Processed frame 1551/1802
Processed frame 1601/1802
Processed frame 1651/1802
Processed frame 1701/1802
Processed frame 1751/1802
Processed frame 1801/1802
Processed frame 1802/1802
Annotated frames saved to: tracked_